In [1]:
# the following copied from https://github.com/openmm/openmmforcefields

# Create an OpenFF Molecule object for benzene from SMILES
from openff.toolkit.topology import Molecule
molecule = Molecule.from_smiles('CO')

# Create the GAFF template generator
from openmmforcefields.generators import GAFFTemplateGenerator
gaff = GAFFTemplateGenerator(molecules=molecule)

# Create an OpenMM ForceField object with AMBER ff14SB and TIP3P with compatible ions
from simtk.openmm.app import ForceField
forcefield = ForceField('amber/ff14SB.xml')

# Register the GAFF template generator
forcefield.registerTemplateGenerator(gaff.generator)

# You can now parameterize an OpenMM Topology object that contains the specified molecule.
# forcefield will load the appropriate GAFF parameters when needed, and antechamber
# will be used to generate small molecule parameters on the fly.
from simtk.openmm.app import PDBFile
pdbfile = PDBFile('equilibrated.pdb')

# system = forcefield.createSystem(pdbfile.topology)


In [2]:
from mdtools import MDSystem
from simtk.unit import *

mdsystem = MDSystem(pdbfile.topology, pdbfile.positions, forcefield)
mdsystem.buildSimulation(posre=True)

In [4]:
from atomsmm.reporters import ExtendedStateDataReporter
from atomsmm.computers import PressureComputer
from simtk.openmm import Platform

current_platform = mdsystem.simulation.context.getPlatform()
reference_platform = Platform.getPlatformByName('Reference')
mdsystem.simulation.reporters.append(ExtendedStateDataReporter(\
                                    f"extended.csv", 500,
                                    molecularPressure=True,
                                    pressureComputer=PressureComputer(mdsystem.simulation.system, 
                                                                      mdsystem.getTopology(),
                                                                      reference_platform)))



In [ ]:
mdsystem.simulation.reporters = mdsystem.simulation.reporters[:-1]

In [ ]:
%%time
mdsystem.equilibrate(0.01*nanoseconds, posre=True)

In [ ]:
mdsystem.save('equilibrated.pdb')

In [3]:
from mdtools import MDSystem
from simtk.unit import *

mdsystem = MDSystem(pdbfile.topology, pdbfile.positions, forcefield)
mdsystem.buildSimulation(filePrefix=f"production1", ensemble='NPT',
                         saveTrajectory=True, trajInterval=500, saveVelocities=True,
                         saveStateData=True, stateDataInterval=500)

In [5]:
%time
mdsystem.simulate(1000*picoseconds)

CPU times: user 7 µs, sys: 3 µs, total: 10 µs
Wall time: 20 µs


In [16]:
mdsystem.save('NPT_1ns.pdb')

In [18]:
mdsystem.buildSimulation(filePrefix=f"production", saveTrajectory=True, 
                         trajInterval=500, saveStateData=True, stateDataInterval=500)

In [19]:
mdsystem.simulate(1000*picoseconds)

In [6]:
import numpy as np

molecular_pressure_data = np.genfromtxt('extended.csv', skip_header=True)

In [15]:
np.mean(molecular_pressure_data[:-200])

-30.458974807156984

In [20]:
for reporter in mdsystem.simulation.reporters:
    try:
        reporter.close()
    except:
        continue